# Загрузка датасетов

In [ ]:
!pip install datasets

In [ ]:
import datasets

In [ ]:
dataset_news = datasets.load_dataset("ag_news")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
dataset_imdb = datasets.load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

**Тексты и метки - news**

In [ ]:
import random

count0, count1, count2, count3 = 0, 0, 0, 0
dataset_short_news = []
for i in range(len(dataset_news['train'])):
  if dataset_news['train'][i]['label'] == 0 and count0 < 2000:
    dataset_short_news.append({'news': dataset_news['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_news['train'][i]['label'] == 1 and count1 < 2000:
    dataset_short_news.append({'news': dataset_news['train'][i]['text'], 'label': 1})
    count1 += 1
  elif dataset_news['train'][i]['label'] == 2 and count2 < 2000:
    dataset_short_news.append({'news': dataset_news['train'][i]['text'], 'label': 2})
    count2 += 1
  elif dataset_news['train'][i]['label'] == 3 and count3 < 2000:
    dataset_short_news.append({'news': dataset_news['train'][i]['text'], 'label': 3})
    count3 += 1

random.shuffle(dataset_short_news)
dataset_news = {'train': dataset_short_news[:6400], 'test': dataset_short_news[6400:]}

In [ ]:
news_X = []
for i in range(len(dataset_news['train'])):
  news_X.append(dataset_news['train'][i]['news'])
for i in range(len(dataset_news['test'])):
  news_X.append(dataset_news['test'][i]['news'])

In [ ]:
news_y = []
for i in range(len(dataset_news['train'])):
  news_y.append(dataset_news['train'][i]['label'])
for i in range(len(dataset_news['test'])):
  news_y.append(dataset_news['test'][i]['label'])

**Тексты и метки - imdb**

In [ ]:
count0, count1 = 0, 0
dataset_short_imdb = []

for i in range(len(dataset_imdb['train'])):
  if dataset_imdb['train'][i]['label'] == 0 and count0 < 4000:
    dataset_short_imdb.append({'text': dataset_imdb['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_imdb['train'][i]['label'] == 1 and count1 < 4000:
    dataset_short_imdb.append({'text': dataset_imdb['train'][i]['text'], 'label': 1})
    count1 += 1

random.shuffle(dataset_short_imdb)
dataset_imdb = {'train': dataset_short_imdb[:6400], 'test': dataset_short_imdb[6400:]}

In [ ]:
imdb_X = []
for i in range(len(dataset_imdb['train'])):
  imdb_X.append(dataset_imdb['train'][i]['text'])
for i in range(len(dataset_imdb['test'])):
  imdb_X.append(dataset_imdb['test'][i]['text'])

In [ ]:
imdb_y = []
for i in range(len(dataset_imdb['train'])):
  imdb_y.append(dataset_imdb['train'][i]['label'])
for i in range(len(dataset_imdb['test'])):
  imdb_y.append(dataset_imdb['test'][i]['label'])

# 1. Обучаем и тестируем LSTM

In [ ]:
!pip install tensorflow keras

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.optimizers import Adam

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

**Токенезируем тексты и подготавливаем метки - news**

In [ ]:
token = Tokenizer()

In [ ]:
token.fit_on_texts(news_X)  # создаём словарь слово - индекс

In [ ]:
X_news = token.texts_to_sequences(news_X)  # предложение в последовательность чисел
X_news = pad_sequences(X_news, maxlen=50)  # выравнивание по длине

In [ ]:
label_encoder = LabelEncoder()
y_news = label_encoder.fit_transform(news_y)
y_news = np.array(y_news)

**Токенезируем тексты и подготавливаем метки - imdb**

In [ ]:
token_imdb = Tokenizer()

In [ ]:
token_imdb.fit_on_texts(imdb_X)

In [ ]:
X_imdb = token_imdb.texts_to_sequences(imdb_X)
X_imdb = pad_sequences(X_imdb, maxlen=200)

In [ ]:
label_encoder_imdb = LabelEncoder()
y_imdb = label_encoder_imdb.fit_transform(imdb_y)
y_imdb = np.array(y_imdb)

**Делим на train/test**

In [ ]:
X_train_news, X_test_news, y_train_news, y_test_news = train_test_split(X_news, y_news, test_size=0.2, random_state=42)

In [ ]:
X_train_imdb, X_test_imdb, y_train_imdb, y_test_imdb = train_test_split(X_imdb, y_imdb, test_size=0.2, random_state=42)

**LSTM для news**

In [ ]:
model_news = Sequential([
    Embedding(input_dim=len(token.word_index) + 1, output_dim=100, input_shape=(50,)),
    SpatialDropout1D(0.2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(4, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model_news.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
model_news.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 50, 100)        │     2,090,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ (None, 50, 100)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │           404 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,171,604 (8.28 MB)

 Trainable params: 2,171,604 (8.28 MB)

 Non-trainable params: 0 (0.00 B)

**Обучение модели**

In [ ]:
history_news = model_news.fit(X_train_news, y_train_news, validation_split=0.1, epochs=8, batch_size=64)

Epoch 1/8
90/90 ━━━━━━━━━━━━━━━━━━━━ 14s 157ms/step - accuracy: 0.8015 - loss: 0.5692 - val_accuracy: 0.8828 - val_loss: 0.4040
Epoch 2/8
90/90 ━━━━━━━━━━━━━━━━━━━━ 19s 137ms/step - accuracy: 0.9201 - loss: 0.2694 - val_accuracy: 0.8906 - val_loss: 0.3562
Epoch 3/8
90/90 ━━━━━━━━━━━━━━━━━━━━ 20s 132ms/step - accuracy: 0.9546 - loss: 0.1670 - val_accuracy: 0.8891 - val_loss: 0.3565
Epoch 4/8
90/90 ━━━━━━━━━━━━━━━━━━━━ 20s 132ms/step - accuracy: 0.9663 - loss: 0.1283 - val_accuracy: 0.8984 - val_loss: 0.3227
Epoch 5/8
90/90 ━━━━━━━━━━━━━━━━━━━━ 11s 122ms/step - accuracy: 0.9779 - loss: 0.0762 - val_accuracy: 0.8891 - val_loss: 0.4192
Epoch 6/8
90/90 ━━━━━━━━━━━━━━━━━━━━ 23s 155ms/step - accuracy: 0.9818 - loss: 0.0695 - val_accuracy: 0.8859 - val_loss: 0.4337
Epoch 7/8
90/90 ━━━━━━━━━━━━━━━━━━━━ 12s 134ms/step - accuracy: 0.9860 - loss: 0.0470 - val_accuracy: 0.8844 - val_loss: 0.3854
Epoch 8/8
90/90 ━━━━━━━━━━━━━━━━━━━━ 22s 156ms/step - accuracy: 0.9886 - loss: 0.0400 - val_accuracy: 0.

In [ ]:
y_pred_news = model_news.predict(X_test_news)
y_pred_news = (y_pred_news > 0.5).astype(int)

50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step


In [ ]:
y_pred_classes = np.argmax(y_pred_news, axis=1)

In [ ]:
print(classification_report(y_test_news, y_pred_classes))

              precision    recall  f1-score   support

           0       0.78      0.82      0.80       397
           1       0.88      0.93      0.91       386
           2       0.84      0.81      0.83       397
           3       0.87      0.81      0.84       420

    accuracy                           0.84      1600
   macro avg       0.84      0.84      0.84      1600
weighted avg       0.84      0.84      0.84      1600



**LSTM для imdb**

In [ ]:
model_imdb = Sequential([
    Embedding(input_dim=len(token_imdb.word_index) + 1, output_dim=100, input_shape=(100,)),
    SpatialDropout1D(0.2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model_imdb.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
model_imdb.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 100, 100)       │     4,031,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_2             │ (None, 100, 100)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 100)            │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,111,901 (15.69 MB)

 Trainable params: 4,111,901 (15.69 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history_imdb = model_imdb.fit(X_train_imdb, y_train_imdb, validation_split=0.1, epochs=8, batch_size=64)

Epoch 1/8
54/54 ━━━━━━━━━━━━━━━━━━━━ 32s 517ms/step - accuracy: 0.5368 - loss: 0.6877 - val_accuracy: 0.6615 - val_loss: 0.6329
Epoch 2/8
54/54 ━━━━━━━━━━━━━━━━━━━━ 39s 481ms/step - accuracy: 0.7921 - loss: 0.5081 - val_accuracy: 0.7682 - val_loss: 0.5162
Epoch 3/8
54/54 ━━━━━━━━━━━━━━━━━━━━ 41s 480ms/step - accuracy: 0.8971 - loss: 0.2879 - val_accuracy: 0.7969 - val_loss: 0.4910
Epoch 4/8
54/54 ━━━━━━━━━━━━━━━━━━━━ 42s 508ms/step - accuracy: 0.9528 - loss: 0.1395 - val_accuracy: 0.7995 - val_loss: 0.5689
Epoch 5/8
54/54 ━━━━━━━━━━━━━━━━━━━━ 26s 478ms/step - accuracy: 0.9767 - loss: 0.0859 - val_accuracy: 0.7969 - val_loss: 0.5961
Epoch 6/8
54/54 ━━━━━━━━━━━━━━━━━━━━ 41s 481ms/step - accuracy: 0.9854 - loss: 0.0526 - val_accuracy: 0.7786 - val_loss: 0.6390
Epoch 7/8
54/54 ━━━━━━━━━━━━━━━━━━━━ 41s 477ms/step - accuracy: 0.9862 - loss: 0.0554 - val_accuracy: 0.8125 - val_loss: 0.7278
Epoch 8/8
54/54 ━━━━━━━━━━━━━━━━━━━━ 41s 476ms/step - accuracy: 0.9929 - loss: 0.0230 - val_accuracy: 0.

In [ ]:
y_pred_imdb = model_imdb.predict(X_test_imdb)
y_pred_imdb = (y_pred_imdb > 0.5).astype(int)

30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 111ms/step


In [ ]:
y_pred_classes_imdb = np.argmax(y_pred_imdb, axis=1)

In [ ]:
print(classification_report(y_test_imdb, y_pred_classes_imdb))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67       479
           1       0.00      0.00      0.00       481

    accuracy                           0.50       960
   macro avg       0.25      0.50      0.33       960
weighted avg       0.25      0.50      0.33       960



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
